In [284]:
import numpy as np
import pandas as pd
import datetime as dt

from src.processing.clustering import LoadShapeCluster
from src.training.sarimax import TrainSARIMAX
from src.training.mlp import DayAheadMLP

# This script is designed to execute ONE (for now) day-ahead forecast given a set of parameters

# SARIMA hyperparameters
location = 'arizona'
numClusters = 3
trendParams = (0,0,0)
seasonalParams = (1,0,0,24)
maxiter = 20

# MLP hyperparameters
lags = range(1,169)
epochs = 50
activation = 'relu'
optimizer='adam'
loss='mse'

# import preprocessed load and covariate data
loads = pd.read_csv('data/processed/'+location+'-loads.csv',index_col=0,date_parser=pd.to_datetime)
covariates = pd.read_csv('data/processed/'+location+'-covariates.csv',index_col=0,date_parser=pd.to_datetime)

# robust method to get today's date in the correct year (based on df.index)
today = dt.datetime.now().date()
testYear = loads.index[(loads.index.month == today.month) & (loads.index.day == today.day)].year[0]
testDate = pd.to_datetime(dt.date(year=testYear, month=today.month, day=today.day))+pd.Timedelta(hours=10)

# apply clustering on previous week of data
clusterTestDf = loads.loc[(testDate-pd.Timedelta(days=7) <= loads.index) & (loads.index < testDate)]
clusterMap, clusterScore = LoadShapeCluster(clusterTestDf,numClusters)

# define test df based on clustering results
df = pd.DataFrame(data=loads.sum(axis=1), index=loads.index, columns=['aggregate'])
for i in range(1,numClusters+1):
    df['cluster'+str(i)] = loads[[k for k in loads.columns if clusterMap[k]==i]].sum(axis=1)
    
# generate MultiIndex to store results, initialize results df
indexMulti = pd.MultiIndex.from_product([list(df.columns),['actual','sarimax','mlp']])
results = pd.DataFrame(index=pd.date_range(start=testDate, freq='H', periods=38), columns=indexMulti)

for cluster in df.columns:
    # grab data for a single cluster, save the test data
    y = df[cluster].copy(deep=True)
    X = covariates.copy(deep=True)
    results[cluster,'actual'] = y.loc[(testDate<=y.index) & (y.index<testDate+pd.Timedelta(hours=38))]

    # use DayAheadSARIMAX helper function to train a fresh model and make a day-ahead forecast
    y_pred_sarimax = DayAheadSARIMAX(endog=y,
                                     exog=X,
                                     date=testDate,
                                     trend=trendParams,
                                     seasonal=seasonalParams,
                                     maxiter=maxiter)
    results[cluster,'sarimax'] = y_pred_sarimax
    
    # use DayAheadMLP helper function to train a fresh model and make a day-ahead forecast
    y_pred_mlp = DayAheadMLP(endog=y,
                             exog=X,
                             date=testDate,
                             lags=lags,
                             epochs=epochs,
                             activation=activation,
                             optimizer=optimizer,
                             loss=loss)
    results[cluster,'mlp'] = y_pred_mlp

results.to_csv('data/results/'+str(today)+'.csv')

Epoch 1/50
7138/7138 [==============================] - 1s 81us/sample - loss: 0.0181
Epoch 2/50
7138/7138 [==============================] - 0s 61us/sample - loss: 0.0045
Epoch 3/50
7138/7138 [==============================] - 0s 68us/sample - loss: 0.0034
Epoch 4/50
7138/7138 [==============================] - 0s 70us/sample - loss: 0.0031
Epoch 5/50
7138/7138 [==============================] - 0s 68us/sample - loss: 0.0029
Epoch 6/50
7138/7138 [==============================] - 1s 87us/sample - loss: 0.0027
Epoch 7/50
7138/7138 [==============================] - 1s 89us/sample - loss: 0.0026
Epoch 8/50
7138/7138 [==============================] - 0s 61us/sample - loss: 0.0025
Epoch 9/50
7138/7138 [==============================] - 1s 86us/sample - loss: 0.0024
Epoch 10/50
7138/7138 [==============================] - 0s 67us/sample - loss: 0.0024
Epoch 11/50
7138/7138 [==============================] - 1s 85us/sample - loss: 0.0024
Epoch 12/50
7138/7138 [=============================

/Users/kevinkoste/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Epoch 1/50
7138/7138 [==============================] - 1s 78us/sample - loss: 0.0184
Epoch 2/50
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0038
Epoch 3/50
7138/7138 [==============================] - 0s 54us/sample - loss: 0.0031
Epoch 4/50
7138/7138 [==============================] - 0s 52us/sample - loss: 0.0027
Epoch 5/50
7138/7138 [==============================] - 0s 52us/sample - loss: 0.0026
Epoch 6/50
7138/7138 [==============================] - 0s 53us/sample - loss: 0.0024
Epoch 7/50
7138/7138 [==============================] - 0s 51us/sample - loss: 0.0024
Epoch 8/50
7138/7138 [==============================] - 0s 57us/sample - loss: 0.0023
Epoch 9/50
7138/7138 [==============================] - 0s 51us/sample - loss: 0.0023
Epoch 10/50
7138/7138 [==============================] - 0s 52us/sample - loss: 0.0023
Epoch 11/50
7138/7138 [==============================] - 0s 54us/sample - loss: 0.0022
Epoch 12/50
7138/7138 [=============================

7138/7138 [==============================] - 0s 53us/sample - loss: 0.0022
Epoch 46/50
7138/7138 [==============================] - 0s 51us/sample - loss: 0.0022
Epoch 47/50
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0022
Epoch 48/50
7138/7138 [==============================] - 0s 54us/sample - loss: 0.0022
Epoch 49/50
7138/7138 [==============================] - 0s 47us/sample - loss: 0.0022
Epoch 50/50
7138/7138 [==============================] - 0s 47us/sample - loss: 0.0021


/Users/kevinkoste/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Epoch 1/50
7138/7138 [==============================] - 0s 69us/sample - loss: 0.0163
Epoch 2/50
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0053
Epoch 3/50
7138/7138 [==============================] - 0s 57us/sample - loss: 0.0044
Epoch 4/50
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0040
Epoch 5/50
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0038
Epoch 6/50
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0036
Epoch 7/50
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0034
Epoch 8/50
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0033
Epoch 9/50
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0032
Epoch 10/50
7138/7138 [==============================] - 0s 59us/sample - loss: 0.0031
Epoch 11/50
7138/7138 [==============================] - 0s 61us/sample - loss: 0.0030
Epoch 12/50
7138/7138 [=============================

In [283]:
# generate MultiIndex to store results, initialize results df
indexMulti = pd.MultiIndex.from_product([list(df.columns),['actual','sarimax','mlp']])
results = pd.DataFrame(index=pd.date_range(start=testDate, freq='H', periods=38), columns=indexMulti)

for cluster in df.columns:
    
    # SARIMAX loop
    y = df[cluster].copy(deep=True)
    X = covariates.copy(deep=True)

    y_train = y.loc[y.index < testDate]
    y_test = y.loc[(testDate <= y.index) & (y.index < testDate+pd.Timedelta(hours=38))]
    X_train = X.loc[X.index < testDate]
    X_test = X.loc[(testDate <= X.index) & (X.index < testDate+pd.Timedelta(hours=38))]

    # train the SARIMAX model for this cluster, make a forecast
    modelFit = TrainSARIMAX(endog=y_train,
                            exog=X_train,
                            trend=trendParams,
                            seasonal=seasonalParams,
                            maxiter=maxiter)
    y_pred_sarimax = modelFit.forecast(steps=38, exog=X_test)
    
    # store the results
    results[cluster,'actual'] = y_test.values
    results[cluster,'sarimax'] = y_pred_sarimax
    
    # MLP loop
    y_pred_mlp = DayAheadMLP(endog=y,
                             exog=X,
                             date=testDate,
                             lags=lags,
                             epochs=epochs,
                             activation=activation,
                             optimizer=optimizer,
                             loss=loss)
    results[cluster,'mlp'] = y_pred_mlp

# results.to_csv('data/results/'+str(today)+'.csv')

ValueError: Unknown activation function:adam

### Work on MLP

In [170]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from src.processing.clustering import LoadShapeCluster

# hyperparameters
location = 'arizona'
numClusters = 3

# import preprocessed load and covariate data
loads = pd.read_csv('data/processed/'+location+'-loads.csv',index_col=0,date_parser=pd.to_datetime)
covariates = pd.read_csv('data/processed/'+location+'-covariates.csv',index_col=0,date_parser=pd.to_datetime)

# robust method to get today's date in the correct year (based on df.index)
today = dt.datetime.now().date()
testYear = loads.index[(loads.index.month == today.month) & (loads.index.day == today.day)].year[0]
testDate = pd.to_datetime(dt.date(year=testYear, month=today.month, day=today.day))+pd.Timedelta(hours=10)

# apply clustering on previous week of data
clusterTestDf = loads.loc[(testDate-pd.Timedelta(days=7) <= loads.index) & (loads.index < testDate)]
clusterMap, clusterScore = LoadShapeCluster(clusterTestDf,numClusters)

# define test df based on clustering results
df = pd.DataFrame(data=loads.sum(axis=1), index=loads.index, columns=['aggregate'])
for i in range(1,numClusters+1):
    df['cluster'+str(i)] = loads[[k for k in loads.columns if clusterMap[k]==i]].sum(axis=1)

In [200]:
# model hyperparameters
lags = range(1,169)
epochs = 100
activation = 'relu'
optimizer='adam'
loss='mse'

# generate MultiIndex to store results, initialize results df
indexMulti = pd.MultiIndex.from_product([list(df.columns),['actual','sarimax','mlp']])
results = pd.DataFrame(index=pd.date_range(start=testDate, freq='H', periods=38), columns=indexMulti)

for cluster in df.columns:
    
    y = pd.DataFrame(df[cluster])
    X = covariates.copy(deep=True)

    # scale y (0,1) on annual min and max, important assumption
    scaler = MinMaxScaler().fit(y)
    y = pd.DataFrame(data=scaler.transform(y),index=y.index,columns=y.columns)

    for i in lags:
        X['t-'+str(i)] = y[cluster].shift(i)
    for j in range(1,38):
        y['t+'+str(j)] = y[cluster].shift(-j)

    # truncate on both sides to remove NaNs
    X.dropna(inplace=True)
    y = y.reindex(X.index, axis=0).dropna()
    X = X.reindex(y.index, axis=0).dropna()

    # train/test split, train range includes all available data up to two days prior to test date
    y_train = y.loc[y.index < testDate-pd.Timedelta(days=2)].values
    y_test = y.loc[y.index == testDate].values
    X_train = X.loc[X.index < testDate-pd.Timedelta(days=2)].values
    X_test = X.loc[X.index == testDate].values

    # set input and hidden layer dimensions
    inputDim = X_train.shape[1]
    hiddenDim = (inputDim-38)//2 + 38

    # define model based on hyperparameters
    model = Sequential()
    model.add(Dense(hiddenDim, activation=activation, input_dim=inputDim))
    model.add(Dense(38))
    model.compile(optimizer=optimizer, loss=loss)

    # fit the model and make a prediction
    model.fit(X_train, y_train, epochs=epochs)
    y_pred = model.predict(X_test, verbose=0)

    # reverse scaling and save results
    y_pred = scaler.inverse_transform(y_pred).flatten()
    y_test = scaler.inverse_transform(y_test).flatten()

    results[cluster,'actual'] = y_test
    results[cluster,'mlp'] = y_pred

Epoch 1/100
7138/7138 [==============================] - 1s 94us/sample - loss: 0.02320s - loss: 0
Epoch 2/100
7138/7138 [==============================] - 0s 65us/sample - loss: 0.0045
Epoch 3/100
7138/7138 [==============================] - 0s 67us/sample - loss: 0.0036
Epoch 4/100
7138/7138 [==============================] - 1s 71us/sample - loss: 0.0032
Epoch 5/100
7138/7138 [==============================] - 0s 65us/sample - loss: 0.0029
Epoch 6/100
7138/7138 [==============================] - 0s 66us/sample - loss: 0.0028
Epoch 7/100
7138/7138 [==============================] - 0s 62us/sample - loss: 0.0027
Epoch 8/100
7138/7138 [==============================] - 0s 64us/sample - loss: 0.0026
Epoch 9/100
7138/7138 [==============================] - 0s 61us/sample - loss: 0.0025
Epoch 10/100
7138/7138 [==============================] - 0s 64us/sample - loss: 0.0024
Epoch 11/100
7138/7138 [==============================] - 0s 66us/sample - loss: 0.0024
Epoch 12/100
7138/7138 [=====

7138/7138 [==============================] - 0s 60us/sample - loss: 0.0016
Epoch 94/100
7138/7138 [==============================] - 0s 61us/sample - loss: 0.00160s - loss: 0.001
Epoch 95/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0016
Epoch 96/100
7138/7138 [==============================] - 0s 58us/sample - loss: 0.0016
Epoch 97/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0016
Epoch 98/100
7138/7138 [==============================] - 0s 59us/sample - loss: 0.0016
Epoch 99/100
7138/7138 [==============================] - 0s 61us/sample - loss: 0.00160s - loss: 0.00
Epoch 100/100
7138/7138 [==============================] - 0s 57us/sample - loss: 0.0016
Epoch 1/100
7138/7138 [==============================] - 0s 69us/sample - loss: 0.0189
Epoch 2/100
7138/7138 [==============================] - 0s 57us/sample - loss: 0.0039
Epoch 3/100
7138/7138 [==============================] - 0s 57us/sample - loss: 0.0031
Epoch 4/100
7138

7138/7138 [==============================] - 0s 60us/sample - loss: 0.0016
Epoch 86/100
7138/7138 [==============================] - 0s 61us/sample - loss: 0.0016
Epoch 87/100
7138/7138 [==============================] - 0s 68us/sample - loss: 0.0016
Epoch 88/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0016
Epoch 89/100
7138/7138 [==============================] - 0s 60us/sample - loss: 0.0016
Epoch 90/100
7138/7138 [==============================] - 0s 64us/sample - loss: 0.0016
Epoch 91/100
7138/7138 [==============================] - 0s 57us/sample - loss: 0.0016
Epoch 92/100
7138/7138 [==============================] - 0s 68us/sample - loss: 0.0015
Epoch 93/100
7138/7138 [==============================] - 0s 63us/sample - loss: 0.0016
Epoch 94/100
7138/7138 [==============================] - 0s 66us/sample - loss: 0.0016
Epoch 95/100
7138/7138 [==============================] - 0s 59us/sample - loss: 0.0016
Epoch 96/100
7138/7138 [=====================

Epoch 78/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0019
Epoch 79/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0020
Epoch 80/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0019
Epoch 81/100
7138/7138 [==============================] - 0s 54us/sample - loss: 0.0019
Epoch 82/100
7138/7138 [==============================] - 0s 60us/sample - loss: 0.0019
Epoch 83/100
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0019
Epoch 84/100
7138/7138 [==============================] - 0s 63us/sample - loss: 0.0018
Epoch 85/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0020
Epoch 86/100
7138/7138 [==============================] - 0s 55us/sample - loss: 0.0018
Epoch 87/100
7138/7138 [==============================] - 0s 61us/sample - loss: 0.0019
Epoch 88/100
7138/7138 [==============================] - 0s 56us/sample - loss: 0.0019
Epoch 89/100
7138/7138 [========

7138/7138 [==============================] - 0s 65us/sample - loss: 0.0022
Epoch 71/100
7138/7138 [==============================] - 1s 70us/sample - loss: 0.0022
Epoch 72/100
7138/7138 [==============================] - 1s 70us/sample - loss: 0.0022
Epoch 73/100
7138/7138 [==============================] - 0s 66us/sample - loss: 0.0022
Epoch 74/100
7138/7138 [==============================] - 0s 64us/sample - loss: 0.0022
Epoch 75/100
7138/7138 [==============================] - 1s 71us/sample - loss: 0.0022
Epoch 76/100
7138/7138 [==============================] - 1s 74us/sample - loss: 0.0022
Epoch 77/100
7138/7138 [==============================] - 0s 70us/sample - loss: 0.0022
Epoch 78/100
7138/7138 [==============================] - 1s 135us/sample - loss: 0.0022
Epoch 79/100
7138/7138 [==============================] - 1s 104us/sample - loss: 0.0022
Epoch 80/100
7138/7138 [==============================] - 1s 86us/sample - loss: 0.0022
Epoch 81/100
7138/7138 [===================

In [210]:
# results['aggregate','actual'].plot()
# results['aggregate','mlp'].plot()

# results['clustersum','mlp'] = results['cluster1','mlp'] + results['cluster2','mlp'] +  results['cluster3','mlp']

results['clustersum','mlp']

mape(results['aggregate','actual'], results['clustersum','mlp'])

mape(results['aggregate','actual'], results['aggregate','mlp'])



1.5909229425745988

In [211]:
def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true))*100